In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, cohen_kappa_score, matthews_corrcoef, roc_curve, roc_auc_score, auc, precision_recall_curve, precision_score, recall_score, f1_score

In [2]:
# Read the stroke_diagnosis.csv file
df = pd.read_csv("CSV\imports\o05_30_percent_filled_dataset.csv")

In [3]:
# Reorder hospital_expire_flag column
column_order = list(df.columns)
column_order.remove('hospital_expire_flag')  # Remove 'flag' from the original position
column_order.append('hospital_expire_flag')  # Add 'flag' to the end

# Reindex the DataFrame
df = df.reindex(columns=column_order)

# Filter icu stay less than 10 days
df = df[df['los'] < 10]

# Filter Time Zone
#df = df[df['Time_Zone'] == 1]

In [4]:
"""
Separate automatic training - test set
keeping patients independent between sets
"""

# Set training percentage. The difference goes to test set
training_percentage = 0.7

# It's already sorted. Just for precaution. Sort by 'subject_id' and 'Time_Zone')
df = df.sort_values(by=['subject_id', 'Time_Zone'])

# Calculate the total number of unique subject IDs
unique_subject_ids = df['subject_id'].nunique()

# Calculate the number of unique subject IDs to include in the training set
train_subject_ids_count = int(training_percentage * unique_subject_ids)

# Initialize variables to track the number of subject IDs included in the training set
subject_ids_in_training = 0

# Initialize empty DataFrames for the training and test sets
train_df = pd.DataFrame(columns=df.columns)
test_df = pd.DataFrame(columns=df.columns)

# Iterate through the sorted DataFrame
for subject_id, subject_data in df.groupby('subject_id'):
    if subject_ids_in_training < train_subject_ids_count:
        # Add this subject's data to the training set
        train_df = pd.concat([train_df, subject_data])
        subject_ids_in_training += 1
    else:
        # Add this subject's data to the test set
        test_df = pd.concat([test_df, subject_data])

# Reset the index of the resulting DataFrames
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

C:\Users\dimop\AppData\Local\Temp\ipykernel_5708\2253970844.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_df = pd.concat([train_df, subject_data])
C:\Users\dimop\AppData\Local\Temp\ipykernel_5708\2253970844.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_df = pd.concat([test_df, subject_data])


In [5]:
"""
Prepaire the training and test set
in encoding level and set labels
"""

# Concatenate train_df and test_df for consistent encoding of categorical variables
combined_df = pd.concat([train_df, test_df], axis=0)

# Encode categorical variables
categorical_cols = ['gender', 'language', 'marital_status', 'race']
combined_df_encoded = pd.get_dummies(combined_df, columns=categorical_cols)

# Convert 'age' column to numeric type
combined_df_encoded['age'] = pd.to_numeric(combined_df_encoded['age'], errors='coerce')

# Convert 'hospital_expire_flag' column to boolean type
combined_df_encoded['hospital_expire_flag'] = combined_df_encoded['hospital_expire_flag'].astype(bool)


# Split the dataframe at the original row index (before concatenation)
combined_df_encoded_train = combined_df_encoded.iloc[:len(train_df)]
combined_df_encoded_test = combined_df_encoded.iloc[len(train_df):]

# Split data into features and target variable again
X_train = combined_df_encoded_train.drop(['row_count', 'subject_id', 'hadm_id', 'Time_Zone', 'hospital_expire_flag'], axis=1)
y_train = combined_df_encoded_train['hospital_expire_flag']
X_test = combined_df_encoded_test.drop(['row_count', 'subject_id', 'hadm_id', 'Time_Zone', 'hospital_expire_flag'], axis=1)
y_test = combined_df_encoded_test['hospital_expire_flag']

In [ ]:
"""
XGBoost HyperParameter
"""

# We can optimize choosing between 'accuracy', 'precision', 'recall', 'f1'
score_metric = 'accuracy'  

# Create a grid search object for XGBoost Classifier
param_grid = {
    'learning_rate': np.arange(0.01, 1.11, 0.10),
    'max_depth': np.arange(1, 11, 1),
    #'reg_lambda': np.arange(0.0, 10.1, 0.1),
    #'reg_alpha': np.arange(0.0, 10.1, 0.1),
    #'gamma': np.arange(0.0, 1.1, 0.1)
}

grid_search = GridSearchCV(estimator=xgb.XGBClassifier(objective='binary:logistic'),
                           param_grid=param_grid,
                           cv=10,  # Number of folds in cross-validation
                           scoring=score_metric,
                           verbose=1,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Predict on test set using the best model
y_pred_best = best_model.predict(X_test)

# Evaluate the model using appropriate classification metrics
accuracy = accuracy_score(y_test, y_pred_best)
precision = precision_score(y_test, y_pred_best)
recall = recall_score(y_test, y_pred_best)
f1 = f1_score(y_test, y_pred_best)

print("Best parameters:", best_params)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Fitting 10 folds for each of 110 candidates, totalling 1100 fits


In [ ]:
"""
XGBoost without HyperParameter
"""

# Define XGBoost model
model = xgb.XGBClassifier()

# Train the model
model.fit(X_train, y_train)

# Predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
kappa = cohen_kappa_score(y_test, predictions)
mcc = matthews_corrcoef(y_test, predictions)
print("Accuracy:", accuracy)
print("\nCohen's Kappa:", kappa)
print("\nMatthews Correlation Coefficient (MCC):", mcc)

# Additional evaluation metrics
print("\n",classification_report(y_test, predictions))

# Most important features

In [ ]:
# Get feature importances
feature_importance = model.feature_importances_

# Create a DataFrame to store feature importances along with their corresponding names
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})

# Sort the DataFrame by feature importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the top N most important features
top_n = 20  # set features number
print(f"Top {top_n} most important features:")
print(feature_importance_df.head(top_n))

In [ ]:
# Get predicted probabilities for the positive class
y_probs = model.predict_proba(X_test)[:, 1]

# Calculate the false positive rate (fpr), true positive rate (tpr), and thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_probs)

# Calculate the area under the curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Calculate precision and recall
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)

# Plot
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()

In [ ]:
# Extract subject_id, hadm_id, and Time_Zone from the test data
df_classifier = test_df[['subject_id', 'hadm_id', 'Time_Zone', 'hospital_expire_flag']].copy()

# Add predictions column
df_classifier['prediction(hospital_expire_flag)'] = predictions

In [ ]:
# Create the ID column by concatenating the values of 'subject_id', 'hadm_id'
df_classifier['subject_&_hadm'] = df_classifier['subject_id'].astype(str) + df_classifier['hadm_id'].astype(str)

# Create column propability of alive (1 for alive 0 for death)
df_classifier['Prediction_of_alive'] = df_classifier['prediction(hospital_expire_flag)'].apply(lambda x: 1 if x == 0 else 0)

# Create column propability of death 1 for death 0 for alive
df_classifier['Prediction_of_death'] = df_classifier['prediction(hospital_expire_flag)'].apply(lambda x: 1 if x == 1 else 0)

# Replace 0 with 1 and 1 with 0 in the 'hospital_expire_flag' column and rename the column
df_classifier['hospital_expire_flag'] = df_classifier['hospital_expire_flag'].replace({0: 1, 1: 0})
df_classifier = df_classifier.rename(columns={'hospital_expire_flag': 'Ground_Truth_Alive'})

# Remove the 'prediction(hospital_expire_flag)' column
df_classifier = df_classifier.drop(columns=['prediction(hospital_expire_flag)'])

In [ ]:
"""
Calculate the probability of Alive and Death
"""

# Create a new column 'denominator' that counts for each 'subject_&_hadm'
df_classifier['Denominator'] = df_classifier.groupby('subject_&_hadm').cumcount() + 1

# Create a new column 'numerator' based on the conditions
#df_classifier['Numerator'] = df_classifier.groupby('subject_&_hadm')['Prediction_of_alive'].transform(
    #lambda x: x.cumsum() if x.iloc[0] != 0 else x.cumsum() - 1
#).replace(-1, 0)



# Create the 'Numerator' column
df_classifier['Numerator'] = df_classifier.groupby('subject_&_hadm')['Prediction_of_alive'].transform(
    lambda x: x.cumsum()
)

# Fill NaN values with 0
df_classifier['Numerator'] = df_classifier['Numerator'].fillna(0)





# Probability of Alive
df_classifier['Probability_of_Alive'] = df_classifier['Numerator'] / df_classifier['Denominator']

# Probability of Death
df_classifier['Probability_of_Death'] = 1 - df_classifier['Probability_of_Alive']


# Repositioning the columns in the DataFrame
columns_to_move = ['subject_&_hadm', 'subject_id', 'hadm_id', 'Time_Zone',
                   'Prediction_of_alive', 'Prediction_of_death',
                   'Ground_Truth_Alive', 'Numerator','Denominator',
                   'Probability_of_Alive', 'Probability_of_Death']
df_classifier = df_classifier[columns_to_move + [col for col in df_classifier.columns if col not in columns_to_move]]


# Export to csv
#df_classifier.to_csv(export_file_path, index=False)

In [ ]:
# Variables
algorithm_label = 'Mimic IV - XGBoost'
xlabel = 'False Positive Rate'
ylabel = 'True Positive Rate'

"""
Assign to variables the values of the first
3, 6, 12 and 24 hour
No1 = first 3 hours
No2 = first 6 hours
...
No16 = first 48 hours
"""
hour1 = df_classifier.loc[df_classifier['Denominator'] == 1] # first 3 hours
hour2 = df_classifier.loc[df_classifier['Denominator'] == 2] # first 6 hours
hour3 = df_classifier.loc[df_classifier['Denominator'] == 4] # first 12 hours
hour4 = df_classifier.loc[df_classifier['Denominator'] == 8] #first 24 hours

"""
Assign to variables values for every hour from
the column Ground Truth Alive and convert from
dataframe to list.
"""

ground_truth_alive1 = hour1["Ground_Truth_Alive"].tolist()
ground_truth_alive2 = hour2["Ground_Truth_Alive"].tolist()
ground_truth_alive3 = hour3["Ground_Truth_Alive"].tolist()
ground_truth_alive4 = hour4["Ground_Truth_Alive"].tolist()

"""
Assign to variables values for every hour from
the column Prob - Alive and convert from
dataframe to list.
"""

prob_alive1 = hour1["Probability_of_Alive"].tolist()
prob_alive2 = hour2["Probability_of_Alive"].tolist()
prob_alive3 = hour3["Probability_of_Alive"].tolist()
prob_alive4 = hour4["Probability_of_Alive"].tolist()

"""
Assign to variables values for every hour from
the column Prob - Dead and convert from
dataframe to list.
"""

prob_dead1 = hour1["Probability_of_Death"].tolist()
prob_dead2 = hour2["Probability_of_Death"].tolist()
prob_dead3 = hour3["Probability_of_Death"].tolist()
prob_dead4 = hour4["Probability_of_Death"].tolist()

"""
To create the Ground Truth Dead per day, I subtract from
the ground_truth_alive of each hour the 1. After the subtraction
I use the abs in order to get the absolute value of it.
So I have the prices without a hitch.
With this process the values of Ground Truth Alive and Ground
Truth Dead are reversed
"""
# Ground Truth Dead per hour
# 1st hour (first 3hours)
ground_truth_dead1=abs(np.subtract(ground_truth_alive1, 1))

# 2nd hour (first 6hours)
ground_truth_dead2=abs(np.subtract(ground_truth_alive2, 1))

# 3rd hour (first 12hours)
ground_truth_dead3=abs(np.subtract(ground_truth_alive3, 1))

# 4th hour (first 24hours)
ground_truth_dead4=abs(np.subtract(ground_truth_alive4, 1))

# Alive

In [ ]:
"""First 3 hours"""

# Your existing code for ROC curve
print("AUC = ", roc_auc_score(ground_truth_alive1, prob_alive1))
fpr1, tpr1, thresholds1 = roc_curve(ground_truth_alive1, prob_alive1)
pyplot.plot(fpr1, tpr1, linestyle='--', label='First 3 Hours')

# Scatter plot for thresholds
for i, threshold in enumerate(thresholds1):
    pyplot.scatter(fpr1[i], tpr1[i], marker='o', color='black', label=f'Threshold = {threshold:.2f}')

pyplot.xlabel(xlabel)
pyplot.ylabel(ylabel)
pyplot.legend()
pyplot.grid(True)
pyplot.title(algorithm_label + " (Alive)")

print("Thresholds          ", thresholds1)
print("False Possitive Rate", fpr1)
print("True Possitive Rate", tpr1)

# Show the plot
pyplot.show()

In [ ]:
"""First 6 hours"""

print("AUC = ", roc_auc_score(ground_truth_alive2, prob_alive2))
fpr2, tpr2, thresholds2 = roc_curve(ground_truth_alive2, prob_alive2)
pyplot.plot(fpr2, tpr2, linestyle='--', label='First 6 Hours')

# Scatter plot for thresholds
for i, threshold in enumerate(thresholds2):
    pyplot.scatter(fpr2[i], tpr2[i], marker='o', color='black', label=f'Threshold = {threshold:.2f}')

pyplot.xlabel(xlabel)
pyplot.ylabel(ylabel)
pyplot.legend()
pyplot.grid(True)
pyplot.title(algorithm_label + " (Alive)")

print("Thresholds          ", thresholds2)
print("False Possitive Rate", fpr2)
print("True Possitive Rate", tpr2)

# Show the plot
pyplot.show()

In [ ]:
"""First 12 hours"""

print("AUC = ", roc_auc_score(ground_truth_alive3, prob_alive3))
fpr3, tpr3, thresholds3 = roc_curve(ground_truth_alive3, prob_alive3)
pyplot.plot(fpr3, tpr3, linestyle='--', label='First 12 Hours')

# Scatter plot for thresholds
for i, threshold in enumerate(thresholds3):
    pyplot.scatter(fpr3[i], tpr3[i], marker='o', color='black', label=f'Threshold = {threshold:.2f}')

pyplot.xlabel(xlabel)
pyplot.ylabel(ylabel)
pyplot.legend()
pyplot.grid(True)
pyplot.title(algorithm_label + " (Alive)")

print("Thresholds          ", thresholds3)
print("False Possitive Rate", fpr3)
print("True Possitive Rate", tpr3)

# Show the plot
pyplot.show()

In [ ]:
"""First 24 hours"""

print("AUC = ", roc_auc_score(ground_truth_alive4, prob_alive4))
fpr4, tpr4, thresholds4 = roc_curve(ground_truth_alive4, prob_alive4)
pyplot.plot(fpr4, tpr4, linestyle='--', label='First 24 Hours')

# Scatter plot for thresholds
for i, threshold in enumerate(thresholds4):
    pyplot.scatter(fpr4[i], tpr4[i], marker='o', color='black', label=f'Threshold = {threshold:.2f}')

pyplot.xlabel(xlabel)
pyplot.ylabel(ylabel)
pyplot.legend()
pyplot.grid(True)
pyplot.title(algorithm_label + " (Alive)")

print("Thresholds          ", thresholds4)
print("False Possitive Rate", fpr4)
print("True Possitive Rate", tpr4)

# Show the plot
pyplot.show()

In [ ]:
# Calculate AUC scores
auc_3_hours = roc_auc_score(ground_truth_alive1, prob_alive1)
auc_6_hours = roc_auc_score(ground_truth_alive2, prob_alive2)
auc_12_hours = roc_auc_score(ground_truth_alive3, prob_alive3)
auc_24_hours = roc_auc_score(ground_truth_alive4, prob_alive4)

# Create a plot
plt.figure(figsize=(12, 8))

# Plot ROC curves with different line styles and colors
plt.plot(fpr1, tpr1, linestyle='-', color='b', label=f'First 3 Hours (AUC = {auc_3_hours:.3f})')
plt.plot(fpr2, tpr2, linestyle='--', color='g', label=f'First 6 Hours (AUC = {auc_6_hours:.3f})')
plt.plot(fpr3, tpr3, linestyle='-.', color='r', label=f'First 12 Hours (AUC = {auc_12_hours:.3f})')
plt.plot(fpr4, tpr4, linestyle=':', color='m', label=f'First 24 Hours (AUC = {auc_24_hours:.3f})')

# Customize the plot
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)
plt.legend(fontsize=12, loc='best')
plt.grid(linestyle='--', alpha=0.7)
plt.title(algorithm_label + " (Alive) for Different Time Intervals", fontsize=25)

# Show the plot
plt.show()